<a href="https://colab.research.google.com/github/ianwolf99/WOLFGPT/blob/main/templates/aistudio_gemini_prompt_chat_b64.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

### Install & import


In [ ]:
!pip install -U -q google-generativeai

In [ ]:
# import necessary modules.
import google.generativeai as genai
import json
import base64
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown

## Set the API key

Add your API_KEY to the secrets manager in the left pannel "🔑".

In [ ]:
from google.colab import userdata

API_KEY=userdata.get('API_KEY')

In [ ]:
# Configure the client library by providing your API key.
genai.configure(api_key=API_KEY)

### Parse the arguments

In [ ]:
model = 'gemini-1.0-pro' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6IndyaXRlIHB5dGhvbiBjb2RlIGZvciB0ZXN0aW5nIHByZXNlbmNlIG9mIGRhdGFiYXNlcyJ9LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOiJgYGBweXRob24gIFxuICAgICMgaW5zdGFuY2VfaWQgPSBcInlvdXItc3Bhbm5lci1pbnN0YW5jZVwiXG4gICAgIyBkYXRhYmFzZV9pZCA9IFwieW91ci1zcGFubmVyLWRiLWlkXCJcblxuICAgIGRhdGFiYXNlX2FkbWluX2NsaWVudCA9IGRhdGFiYXNlX2FkbWluX3YxLkRhdGFiYXNlQWRtaW5DbGllbnQoKVxuXG4gICAgZGF0YWJhc2VfcGF0aCA9IGRhdGFiYXNlX2FkbWluX2NsaWVudC5kYXRhYmFzZV9wYXRoKGluc3RhbmNlX2lkLCBkYXRhYmFzZV9pZClcbiAgICB0cnk6XG4gICAgICAgIGRhdGFiYXNlID0gZGF0YWJhc2VfYWRtaW5fY2xpZW50LmdldF9kYXRhYmFzZShuYW1lPWRhdGFiYXNlX3BhdGgpXG4gICAgICAgIHByaW50KFwiRGF0YWJhc2Uge30gaXMgZm91bmQuXCIuZm9ybWF0KGRhdGFiYXNlLm5hbWUpKVxuICAgICAgICByZXR1cm4gZGF0YWJhc2VcbiAgICBleGNlcHQgZ29vZ2xlLmFwaV9jb3JlLmV4Y2VwdGlvbnMuTm90Rm91bmQ6XG4gICAgICAgIHByaW50KFwiRGF0YWJhc2Uge30gaXMgbm90IGZvdW5kLlwiLmZvcm1hdChkYXRhYmFzZV9pZCkpXG4gICAgICAgIHJldHVybiBOb25lICBcbmBgYCJ9LHsicm9sZSI6InVzZXIiLCJwYXJ0cyI6IndyaXRlIGEgZnVsbCB0dXRvcmlhbCBvZiBob3cgdG8gdXNlIHdpcmVzaGFyayJ9LHsicm9sZSI6Im1vZGVsIiwicGFydHMiOiIqKldpcmVzaGFyayBUdXRvcmlhbCoqXG5cbioqSW50cm9kdWN0aW9uKipcblxuV2lyZXNoYXJrIGlzIGEgcG93ZXJmdWwgbmV0d29yayBwcm90b2NvbCBhbmFseXplciB0aGF0IGFsbG93cyB5b3UgdG8gY2FwdHVyZSwgaW5zcGVjdCwgYW5kIGFuYWx5emUgbmV0d29yayB0cmFmZmljLiBJdCBpcyBhbiBlc3NlbnRpYWwgdG9vbCBmb3IgbmV0d29yayBlbmdpbmVlcnMsIHNlY3VyaXR5IGFuYWx5c3RzLCBhbmQgYW55b25lIGVsc2Ugd2hvIG5lZWRzIHRvIHVuZGVyc3RhbmQgaG93IG5ldHdvcmtzIG9wZXJhdGUuXG5cbioqSW5zdGFsbGluZyBXaXJlc2hhcmsqKlxuXG5XaXJlc2hhcmsgaXMgYXZhaWxhYmxlIGZvciBXaW5kb3dzLCBtYWNPUywgYW5kIExpbnV4LiBZb3UgY2FuIGRvd25sb2FkIHRoZSBsYXRlc3QgdmVyc2lvbiBmcm9tIHRoZSBXaXJlc2hhcmsgd2Vic2l0ZTogaHR0cHM6Ly93d3cud2lyZXNoYXJrLm9yZy9kb3dubG9hZC5odG1sXG5cbioqQ2FwdHVyaW5nIFRyYWZmaWMqKlxuXG5PbmNlIHlvdSBoYXZlIGluc3RhbGxlZCBXaXJlc2hhcmssIHlvdSBjYW4gc3RhcnQgY2FwdHVyaW5nIHRyYWZmaWMuIFRvIGRvIHRoaXMsIGNsaWNrIG9uIHRoZSBcIkNhcHR1cmVcIiBtZW51IGFuZCBzZWxlY3QgXCJJbnRlcmZhY2VzXCIuIFRoaXMgd2lsbCBvcGVuIGEgbGlzdCBvZiBhbGwgdGhlIG5ldHdvcmsgaW50ZXJmYWNlcyBvbiB5b3VyIGNvbXB1dGVyLiBTZWxlY3QgdGhlIGludGVyZmFjZSB0aGF0IHlvdSB3YW50IHRvIGNhcHR1cmUgdHJhZmZpYyBvbiBhbmQgY2xpY2sgXCJTdGFydFwiLlxuXG5XaXJlc2hhcmsgd2lsbCBub3cgc3RhcnQgY2FwdHVyaW5nIHRyYWZmaWMgb24gdGhlIHNlbGVjdGVkIGludGVyZmFjZS4gWW91IGNhbiBzZWUgdGhlIGNhcHR1cmVkIHRyYWZmaWMgaW4gdGhlIG1haW4gV2lyZXNoYXJrIHdpbmRvdy5cblxuKipJbnNwZWN0aW5nIFRyYWZmaWMqKlxuXG5PbmNlIHlvdSBoYXZlIGNhcHR1cmVkIHNvbWUgdHJhZmZpYywgeW91IGNhbiBzdGFydCBpbnNwZWN0aW5nIGl0LiBXaXJlc2hhcmsgcHJvdmlkZXMgYSB2YXJpZXR5IG9mIHRvb2xzIHRvIGhlbHAgeW91IGRvIHRoaXMsIGluY2x1ZGluZzpcblxuKiAqKlBhY2tldCBMaXN0OioqIFRoZSBwYWNrZXQgbGlzdCBzaG93cyBhIGxpc3Qgb2YgYWxsIHRoZSBwYWNrZXRzIHRoYXQgaGF2ZSBiZWVuIGNhcHR1cmVkLiBZb3UgY2FuIGNsaWNrIG9uIGEgcGFja2V0IHRvIHZpZXcgaXRzIGRldGFpbHMuXG4qICoqUGFja2V0IERldGFpbHM6KiogVGhlIHBhY2tldCBkZXRhaWxzIHBhbmUgc2hvd3MgdGhlIGNvbnRlbnRzIG9mIHRoZSBzZWxlY3RlZCBwYWNrZXQuIFlvdSBjYW4gdXNlIHRoaXMgcGFuZSB0byB2aWV3IHRoZSBwYWNrZXQncyBoZWFkZXJzLCBwYXlsb2FkLCBhbmQgb3RoZXIgaW5mb3JtYXRpb24uXG4qICoqRmlsdGVyIEJhcjoqKiBUaGUgZmlsdGVyIGJhciBhbGxvd3MgeW91IHRvIGZpbHRlciB0aGUgY2FwdHVyZWQgdHJhZmZpYy4gWW91IGNhbiB1c2UgdGhlIGZpbHRlciBiYXIgdG8gb25seSBzaG93IHBhY2tldHMgdGhhdCBtZWV0IGNlcnRhaW4gY3JpdGVyaWEsIHN1Y2ggYXMgcGFja2V0cyBmcm9tIGEgc3BlY2lmaWMgSVAgYWRkcmVzcyBvciBwb3J0LlxuXG4qKkFuYWx5emluZyBUcmFmZmljKipcblxuT25jZSB5b3UgaGF2ZSBpbnNwZWN0ZWQgdGhlIHRyYWZmaWMsIHlvdSBjYW4gc3RhcnQgYW5hbHl6aW5nIGl0LiBXaXJlc2hhcmsgcHJvdmlkZXMgYSB2YXJpZXR5IG9mIHRvb2xzIHRvIGhlbHAgeW91IGRvIHRoaXMsIGluY2x1ZGluZzpcblxuKiAqKlN0YXRpc3RpY3M6KiogVGhlIHN0YXRpc3RpY3MgcGFuZSBzaG93cyBhIHZhcmlldHkgb2Ygc3RhdGlzdGljcyBhYm91dCB0aGUgY2FwdHVyZWQgdHJhZmZpYywgc3VjaCBhcyB0aGUgbnVtYmVyIG9mIHBhY2tldHMsIHRoZSBhdmVyYWdlIHBhY2tldCBzaXplLCBhbmQgdGhlIG1vc3QgY29tbW9uIHByb3RvY29scy5cbiogKipHcmFwaHM6KiogVGhlIGdyYXBocyBwYW5lIHNob3dzIGEgdmFyaWV0eSBvZiBncmFwaHMgdGhhdCBjYW4gaGVscCB5b3UgdmlzdWFsaXplIHRoZSBjYXB0dXJlZCB0cmFmZmljLiBGb3IgZXhhbXBsZSwgeW91IGNhbiB1c2UgdGhlIGdyYXBocyBwYW5lIHRvIHNlZSBob3cgdGhlIHRyYWZmaWMgdm9sdW1lIGNoYW5nZXMgb3ZlciB0aW1lIG9yIGhvdyB0aGUgdHJhZmZpYyBpcyBkaXN0cmlidXRlZCBhY3Jvc3MgZGlmZmVyZW50IHByb3RvY29scy5cblxuKipDb25jbHVzaW9uKipcblxuV2lyZXNoYXJrIGlzIGEgcG93ZXJmdWwgdG9vbCB0aGF0IGNhbiBiZSB1c2VkIHRvIGNhcHR1cmUsIGluc3BlY3QsIGFuZCBhbmFseXplIG5ldHdvcmsgdHJhZmZpYy4gSXQgaXMgYW4gZXNzZW50aWFsIHRvb2wgZm9yIG5ldHdvcmsgZW5naW5lZXJzLCBzZWN1cml0eSBhbmFseXN0cywgYW5kIGFueW9uZSBlbHNlIHdobyBuZWVkcyB0byB1bmRlcnN0YW5kIGhvdyBuZXR3b3JrcyBvcGVyYXRlLiJ9XQ==' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MC45LCJ0b3BfcCI6MSwidG9wX2siOjEsIm1heF9vdXRwdXRfdG9rZW5zIjoyMDQ4LCJzdG9wX3NlcXVlbmNlcyI6W119' # @param {isTemplate: true}
safety_settings_b64 = 'W3siY2F0ZWdvcnkiOiJIQVJNX0NBVEVHT1JZX0hBUkFTU01FTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfSEFURV9TUEVFQ0giLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfU0VYVUFMTFlfRVhQTElDSVQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn0seyJjYXRlZ29yeSI6IkhBUk1fQ0FURUdPUllfREFOR0VST1VTX0NPTlRFTlQiLCJ0aHJlc2hvbGQiOiJCTE9DS19NRURJVU1fQU5EX0FCT1ZFIn1d' # @param {isTemplate: true}
user_input_b64 = '' # @param {isTemplate: true}

contents = json.loads(base64.b64decode(contents_b64))
generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))
user_input = base64.b64decode(user_input_b64).decode()
stream = False

In [ ]:
contents

[{'role': 'user', 'parts': [{'text': 'hello'}]},
 {'role': 'model', 'parts': [{'text': 'Hello! How can I assist you today?'}]}]

In [ ]:
generation_config

{}

In [ ]:
safety_settings

{}

In [ ]:
user_input

'How does electricity work?'

### Call the API

In [ ]:
# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

chat = gemini.start_chat(history=contents)

response = chat.send_message(
    user_input,
    stream=stream)

In [ ]:
display(Markdown(response.text))

Electricity is the flow of electric charge. It is a fundamental part of nature and is responsible for a wide range of phenomena, from lightning and static electricity to the operation of electronic devices.

**Basic Principles of Electricity:**

1. **Electric Charge:** Matter is made up of atoms, which have a positively charged nucleus surrounded by negatively charged electrons. The movement of these charged particles creates electric currents.

2. **Electric Field:** Electric charges create an electric field around them. The strength and direction of this field depend on the amount and type of charge.

3. **Electric Potential:** Electric potential is the amount of electrical energy stored in a region of space. It is measured in volts (V).

4. **Electric Current:** Electric current is the flow of electric charge through a conductor. It is measured in amperes (A).

**How Electricity Works:**

1. **Generation of Electricity:** Electricity can be generated in various ways, including:
   - **Mechanical Energy:** Generators convert mechanical energy into electrical energy by spinning a conductor in a magnetic field. This is the principle behind power plants that use turbines driven by steam, water, or wind.
   - **Chemical Energy:** Batteries generate electricity through chemical reactions.
   - **Solar Energy:** Solar cells convert sunlight directly into electricity using the photovoltaic effect.

2. **Transmission of Electricity:** Electricity is transmitted over long distances through power lines. These lines are made of conductive materials, such as copper or aluminum, which allow electrons to flow easily.

3. **Distribution of Electricity:** Once electricity reaches populated areas, it is distributed to homes, businesses, and other consumers through a network of local power lines and transformers.

4. **Utilization of Electricity:** Electricity is used to power various devices and appliances by converting electrical energy into other forms of energy, such as light, heat, or motion.

The flow of electricity in a circuit is driven by the difference in electric potential between two points. When a complete circuit is formed, electrons can flow from a region of higher potential to a region of lower potential, creating an electric current.

Electricity is a versatile form of energy that has revolutionized modern society. It is used in a wide range of applications, from lighting and heating to powering computers and transportation systems.

In [ ]:
response.prompt_feedback

safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}

In [ ]:
response.candidates

[index: 0
content {
  parts {
    text: "Electricity is the flow of electric charge. It is a fundamental part of nature and is responsible for a wide range of phenomena, from lightning and static electricity to the operation of electronic devices.\n\n**Basic Principles of Electricity:**\n\n1. **Electric Charge:** Matter is made up of atoms, which have a positively charged nucleus surrounded by negatively charged electrons. The movement of these charged particles creates electric currents.\n\n2. **Electric Field:** Electric charges create an electric field around them. The strength and direction of this field depend on the amount and type of charge.\n\n3. **Electric Potential:** Electric potential is the amount of electrical energy stored in a region of space. It is measured in volts (V).\n\n4. **Electric Current:** Electric current is the flow of electric charge through a conductor. It is measured in amperes (A).\n\n**How Electricity Works:**\n\n1. **Generation of Electricity:** Electr